In [1]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision import transforms

import os
import sys
from pathlib import Path
modules_path = os.path.abspath(".")
if modules_path not in sys.path:
    sys.path.append(modules_path)

from data.FloodNet import FloodNetDataset

In [2]:
device = (
    "cuda" if torch.cuda.is_available()
    else "mps" if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

Using cuda device


In [3]:
transform = transforms.Compose([
    transforms.Resize((28, 28)),  # Resize the image to 224x224 (adjust to fit your model)
    transforms.ToTensor(),  # Convert image to a PyTorch Tensor
    transforms.Normalize(mean=[0.485, 0.456, 0.406],  # Normalize with ImageNet mean and std
                         std=[0.229, 0.224, 0.225])
])

In [4]:
# Create train dataset
train_images_dir = Path("./data/FloodNet_dataset/train/image")
train_csv_file = Path("./data/flood_train_rel_paths.csv")
train_data = FloodNetDataset(annotations_file=train_csv_file, img_dir=train_images_dir, transform=transform)

# Create test dataset
test_images_dir = Path("./data/FloodNet_dataset/test/image")
test_csv_file = Path("./data/flood_test_rel_paths.csv")
test_data = FloodNetDataset(annotations_file=test_csv_file, img_dir=test_images_dir, transform=transform)

In [5]:
batch_size = 64

train_dataloader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
test_dataloader = DataLoader(test_data, batch_size=batch_size, shuffle=False)

In [ ]:
for X, y in train_dataloader:
    print(f"Shape of train X [N, C, H, W]: {X.shape}")
    print(f"Shape of train y: {y.shape} {y.dtype}")
    break

for X, y in test_dataloader:
    print(f"Shape of test X [N, C, H, W]: {X.shape}")
    print(f"Shape of test y: {y.shape} {y.dtype}")
    break

In [6]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28*3, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 1)
        )

    def forward(self, x):
        # x = self.flatten(x)
        # logits = self.linear_relu_stack(x)
        # return logits
        return self.linear_relu_stack(self.flatten(x))

In [7]:
model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=2352, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=1, bias=True)
  )
)


In [8]:
loss_fn = nn.BCEWithLogitsLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)

In [9]:
def train(dataloader: DataLoader, model, loss_fn, optimizer):
    print("Training...")
    size = len(dataloader.dataset)
    current = 0
    model.train()
    # print(f"num batches: {len(dataloader)}")
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device).float().unsqueeze(1)
        # compute prediction error
        optimizer.zero_grad()

        pred = model(X)

        loss = loss_fn(pred, y)

        # backpropagation
        loss.backward()
        optimizer.step()

        # if batch % 100 == 0:
        loss, current = loss.item(), current + len(X)
        print(f"batch: {batch}, loss: {loss:>7f} [{current:>5d}/{size:>5d}]")

In [11]:
def test(dataloader: DataLoader, model, loss_fn):
    print("Testing...")
    size = len(dataloader.dataset)
    print(f"size: {size}")
    num_batches = len(dataloader)
    print(f"num batches: {num_batches}")
    model.eval()
    test_loss, correct = 0, 0
    with torch.inference_mode():
        for X, y in dataloader:
            print(f"X len: {len(X)}, y len: {len(y)}")
            X, y = X.to(device), y.to(device)
            # y = y.float()

            pred = model(X)
            print(f"pred: {pred}")
            # y = y.unsqueeze(1)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")

In [10]:
def test2(dataloader: DataLoader, model):
    print("Testing...")
    size = len(dataloader.dataset)
    print(f"size: {size}")
    num_batches = len(dataloader)
    print(f"num batches: {num_batches}")
    model.eval()
    correct = 0
    total = 0
    with torch.inference_mode():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)

            outputs = model(X)
            preds = torch.sigmoid(outputs) > 0.5
            correct += (preds.squeeze().long() == y).sum().item()
            total += y.size(0)
    # print(f"Test error: \n Accuracy: {(100*correct):>0.1f}%, avg loss: {test_loss:>8f} \n")
    print(f"Test error: \n Accuracy: {correct / total * 100:.2f}%")

In [11]:
epochs = 5

for epoch in range(epochs):
    print(f"-------------- Epoch {epoch+1} --------------")
    train(train_dataloader, model, loss_fn, optimizer)
    # test(test_dataloader, model, loss_fn)
    test2(test_dataloader, model)
print("Done!")

-------------- Epoch 1 --------------
Training...
batch: 0, loss: 0.665318 [   64/ 1445]
batch: 1, loss: 0.673140 [  128/ 1445]
batch: 2, loss: 0.472500 [  192/ 1445]
batch: 3, loss: 0.490387 [  256/ 1445]


KeyboardInterrupt: 

In [ ]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

In [ ]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth", weights_only=True))

In [35]:
model.eval()
x, y = test_data[0][0], test_data[0][1]
x = x.to(device).unsqueeze(0)
with torch.no_grad():
    pred = model(x)
    prob = torch.sigmoid(pred).item()
    prediction = 1 if prob > 0.5 else 0
    print(f"Predicted class: {prediction}, actual: {y} (probability: {prob:.2f})")

Predicted class: 1, actual: 1 (probability: 0.97)


In [12]:
from torchinfo import summary
summary(model, input_size=[1, 3, 28, 28])

Layer (type:depth-idx)                   Output Shape              Param #
NeuralNetwork                            [1, 1]                    --
├─Flatten: 1-1                           [1, 2352]                 --
├─Sequential: 1-2                        [1, 1]                    --
│    └─Linear: 2-1                       [1, 512]                  1,204,736
│    └─ReLU: 2-2                         [1, 512]                  --
│    └─Linear: 2-3                       [1, 512]                  262,656
│    └─ReLU: 2-4                         [1, 512]                  --
│    └─Linear: 2-5                       [1, 1]                    513
Total params: 1,467,905
Trainable params: 1,467,905
Non-trainable params: 0
Total mult-adds (Units.MEGABYTES): 1.47
Input size (MB): 0.01
Forward/backward pass size (MB): 0.01
Params size (MB): 5.87
Estimated Total Size (MB): 5.89